# Portfolio 
## Andrea Mejia 
#### (858) 466-9040 
#### Email me at betterbiz.drea@gmail.com
This notebook to demonstrate some of my skillsets in python. This Project will focus on demonstrating data mining, web scraping, data analytics and basic machine learning. 

I will be demonstrating an analysis of twitter post. 

In [1]:
#Imports
import numpy as np
import pandas as pd 
#import that social networking service scraper
import snscrape.modules.twitter as sntwitter
import snscrape
# Import nltk package 
# NLTK provides support for a wide variety of text processing tasks: 
# tokenization, stemming, proper name identification, part of speech identification, etc. 
#   PennTreeBank word tokenizer 
#   English language stopwords
import nltk
from nltk.tokenize import word_tokenize
from nltk.corpus import stopwords
nltk.download('vader_lexicon') # this will help us analyze tweet emotions
# scikit-learn imports
#   TF-IDF Vectorizer that first removes widely used words in the dataset and then transforms test data
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
# snsscrape will scrape twitter
import snscrape.modules.twitter as sntwitter
import snscrape as sn
import snscrape.modules.twitter as sntwitter
# import re for regular expression
import re

# seaborn for plotting
import seaborn as sns
sns.set(font_scale=1.2, style="white")
from datetime import datetime, timedelta

# import matplotlib for plotting
import matplotlib as mpl
import matplotlib.pyplot as plt
import matplotlib.style as style
# set plotting size parameter
plt.rcParams['figure.figsize'] = (12, 6)

# improve resolution
%config InlineBackend.figure_format ='retina'

from nltk.stem import PorterStemmer
from nltk.probability import FreqDist
import string
import warnings
warnings.filterwarnings('ignore')

[nltk_data] Downloading package vader_lexicon to
[nltk_data]     /Users/drea/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!


In [2]:
# download stopwords & punkt
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /Users/drea/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /Users/drea/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

# Getting the Data

In [10]:

    

query = str
def twitter_search(query):
    """ This function provides a convient way to retrieve twitter data and put it in a dataframe"""

    query = query
    tweets = []
    limit = 10000


    for tweet in sntwitter.TwitterSearchScraper(query).get_items():
        
        if len(tweets) == limit: 
            break
        else:
            tweets.append([tweet.date,tweet.user.username,tweet.content,])
        
    df = pd.DataFrame(tweets, columns = ['Date','User','content'])
    return df

In [12]:
test_df = twitter_search(' (@padres) until:2023-12-31 since:2023-01-01 -filter:replies')
test_df_copy = test_df.copy() #copy of test_df for sentiment analysis
test_df.head()




,Date,User,content
0,2023-01-24 22:30:28+00:00,Padres,"5 years ago, @THoffman51 got the HOF call... h..."
1,2023-01-24 22:18:59+00:00,ColeThomas0,a zillion/10 @Padres
2,2023-01-24 21:44:16+00:00,hector8c,"One month later, still hurts @Padres"
3,2023-01-24 21:27:57+00:00,10NewsCoronel,.@MLB investigating @WhiteSox pitcher &amp; fo...
4,2023-01-24 21:19:56+00:00,TheKingSource,Yes and zero offers. You'd think a time whiteo...


In [ ]:
#let's download it so we don't have to run the cell everytime. 
test_df_copy.to_csv('Test_Content_dataset', index=False)

In [13]:
test_df.shape

(895, 3)